In [1]:
%cd ~/SSMuLA

/disk2/fli/SSMuLA


In [71]:
%load_ext blackcellmagic

In [2]:
!python sandbox/esmif/score_log_likelihoods.py sandbox/esmif/5YH2.pdb \
  sandbox/esmif/5YH2_mutated_seqs.fasta --chain C \
  --outpath sandbox/esmif/5YH2_mutated_seqs_scores.csv

Traceback (most recent call last):
  File "/disk2/fli/SSMuLA/sandbox/esmif/score_log_likelihoods.py", line 12, in <module>
    from biotite.sequence.io.fasta import FastaFile, get_sequences
ModuleNotFoundError: No module named 'biotite'


In [46]:
import warnings

from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1
from Bio import SeqIO, pairwise2
from Bio.pairwise2 import format_alignment

from SSMuLA.landscape_global import LIB_INFO_DICT

warnings.filterwarnings("ignore")

In [47]:
def pdb2seq(pdb_file_path: str, chain_id: str = "A") -> str:

    """
    A function for extracting chain in string format from pdb

    Args:
    - pdb_file_path: str, 
    - chain_id: str = "A"
    """

    chains = {record.id: record.seq for record in SeqIO.parse(pdb_file_path, 'pdb-atom')}
    
    return str(chains[[chain for chain in chains.keys() if chain_id in chain][0]])

In [48]:
for lib in LIB_INFO_DICT.keys():
    print(lib)
    if "TrpB" in lib:
        protein = "TrpB"
    else:
        protein = lib

    print(f"data/{protein}/{protein}.pdb")

    seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq
    if lib == "DHFR":
        seq = str(seq.translate())
    else:
        seq = str(seq)

    pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

    global_alignments = pairwise2.align.globalxx(seq, pdb_seq)

    print(len(seq), len(pdb_seq))
    
    # pdb has more than fasta should only be for dhfr
    if len(seq) < len(pdb_seq):
        
        
        
        
        
        

DHFR
data/DHFR/DHFR.pdb
73 159
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVT MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR
Global Alignment:
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVT--------------------------------------------------------------------------------------
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||                                                                                      
MISLIAALAVDRVIGMENAMPWNLPADLAWFKRNTLNKPVIMGRHTWESIGRPLPGRKNIILSSQPGTDDRVTWVKSVDEAIAACGDVPEIMVIGGGRVYEQFLPKAQKLYLTHIDAEVEGDTHFPDYEPDDWESVFSEFHDADAQNSHSYCFEILERR
  Score=73

ParD2
data/ParD2/ParD2.pdb
91 87
MANVEKMSVAVTPQQAAVMREAVEAGEYATASEIVREAVRDWLAKRELREAEAERLRKAWIEGLESGPFAPFDIEDIKQKARSRLVDAIKK NVEKMSVAVTPQQAAVMREAVEAGEYATASEIVREAVRDWLAKRELREAEAERLRKAWIEGLESGPFAPFDIEDIKQKARSRLVDAI
Global Align

In [55]:
global_alignments[0].seqA

'MKGYFGPYGGQYVPEILMGALEELEAAYEGIMKDESFWKEFNDLLRDYAGRPTPLYFARRLSEKYGARVYLKREDLLHTGAHKINNAIGQVLLAKLMGKTRIIAETGAGQHGVATATAAALFGMECVIYMGEEDTIRQKLNVERMKLLGAKVVPVKSGSRTLKDAIDEALRDWITNLQTTYYVFGSVVGPHPYPIIVRNFQKVIGEETKKQIPEKEGRLPDYIVACVSGGSNAAGIFYPFIDSGVKLIGVEAGGEGLETGKHAASLLKGKIGYLHGSKTFVLQDDWGQVQVSHSVSAGLDYSGVGPEHAYWRETGKVLYDAVTDEEALDAFIELSRLEGIIPALESSHALAYLKKINIKGKVVVVNLSGRGDKDLESVLNHPYVRERIRL'

In [56]:
global_alignments[0].seqB

'--GYFGPYGGQYVPEILMGALEELEAAYEGIMKDESFWKEFNDLLRDYAGRPTPLYFARRLSEKYGARVYLKREDLLHTGAHKINNAIGQVLLAKLMGKTRIIAETGAGQHGVATATAAALFGMECVIYMGEEDTIRQKLNVERMKLLGAKVVPVKSGSRTLKDAIDEALRDWITNLQTTYYVFGSVVGPHPYPIIVRNFQKVIGEETKKQIPEKEGRLPDYIVACVSGGSNAAGIFYPFIDSGVKLIGVEAGGEGLETGKHAASLLKGKIGYLHGSKTFVLQDDWGQVQVSHSVSAGLDYSGVGPEHAYWRETGKVLYDAVTDEEALDAFIELSRLEGIIPALESSHALAYLKKINIKGKVVVVNLSGRGDKDLESVLNHPYVRERIRL'

In [66]:
def find_missing_str(longer: str, shorter: str) -> [str, str]:
    """
    A function 
    """
    # Find the start index of the shorter in the longer string
    start_index = longer.find(shorter)
    
    # If the shorter is not found, return the longer string as the "missing" part
    if start_index == -1:
        return longer, ''
    
    # Find the end index of the shorter
    end_index = start_index + len(shorter)
    
    # Extract parts of the longer string that are not the shorter
    part_before = longer[:start_index]
    part_after = longer[end_index:]
    
    return part_before, part_after

In [67]:
ev_esm_dir = "ev_esm2"
csv_path = f"{ev_esm_dir}/{lib}/{lib}.csv"

if "TrpB" in lib:
    protein = "TrpB"
else:
    protein = lib
    
seq = SeqIO.read(f"data/{protein}/{protein}.fasta", "fasta").seq
if lib == "DHFR":
    seq = str(seq.translate())
else:
    seq = str(seq)

pdb_seq = pdb2seq(f"data/{protein}/{protein}.pdb", "A")

global_alignments = pairwise2.align.globalxx(seq, pdb_seq)

df = pd.read_csv(df_path)

for col in ["muts", "seq"]:
    if col not in df.columns:
        raise ValueError(f"{col} column not found")

fasta = csv_path.replace(".csv", ".fasta")

# pdb has more than fasta should only be for dhfr
if len(seq) < len(pdb_seq):
    part_before, part_after = find_missing_str(longer=pdb_seq, shorter=seq)
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{part_before+seq+part_after}\n")
elif len(seq) == len(pdb_seq):
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq}\n")
else:
    part_before, part_after = find_missing_str(longer=seq, shorter=pdb_seq)
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq[len(part_before):len(seq)-len(part_after)]}\n")



TypeError: find_missing_parts() got an unexpected keyword argument 'longer'

In [ ]:

def csv2fasta(csv: str) -> None:
    """
    A function for converting a csv file to a fasta file
    ie /disk2/fli/SSMuLA/ev_esm2/DHFR/DHFR.csv

    """
    df = pd.read_csv(csv)

    for col in ["muts", "seq"]:
        if col not in df.columns:
            raise ValueError(f"{col} column not found")

    fasta = csv.replace(".csv", ".fasta")
    with open(fasta, "w") as f:
        for mut, seq in zip(df["muts"].values, df["seq"].values):
            f.write(f">{mut}\n{seq}\n")
